# (1) Setup

## Imports

In [1]:
from bs4 import BeautifulSoup
from IPython.display import display, HTML

# Settings

In [7]:
# 🔴 SETTING 1: Player Name Format
# Set to True to use the full name extracted from the URL (e.g., "Rafael Nadal")
# Set to False to use the abbreviated name from the link text (e.g., "R. Nadal")
USE_FULL_NAME = True 

# 🔴 SETTING 2: Tournament Size (28, 32, 64, or 128)
PLAYER_COUNT = 128 

# 🟢 SETTING 3: Best-of Format
# Determines the maximum number of sets in a match (e.g., 5 for Grand Slam men's, 3 other events)
BEST_OF_SETS = 5

## Helper Functions

In [8]:
def format_player_name(url):
    """Extracts player name from URL and formats it to Title Case (e.g., 'rafael-nadal' -> 'Rafael Nadal')."""
    if not url or "/en/players/" not in url:
        return ""
    
    # Get the part between '/en/players/' and the player ID '/n409/overview'
    try:
        name_part = url.split("/en/players/")[1].split("/")[0]
        # Replace hyphens with spaces and capitalize the first letter of each word
        formatted_name = " ".join(word.capitalize() for word in name_part.split("-"))
        return formatted_name
    except IndexError:
        return ""

## Round Setups

In [9]:
# Define matches per round and round labels dynamically based on PLAYER_COUNT
if PLAYER_COUNT == 128:
    wiki_template = "Bracket/128"
    matches_per_round = {
        1: 64,  # Round 1
        2: 32,  # Round 2
        3: 16,  # Round 3
        4: 8,   # Round 4
        5: 4,   # Quarterfinals
        6: 2,   # Semifinals
        7: 1    # Grand Final
    }
    round_labels = {
        1: "Round 1",
        2: "Round 2",
        3: "Round 3",
        4: "Round 4",
        5: "Quarterfinals",
        6: "Semifinals",
        7: "Grand Final"
    }
elif PLAYER_COUNT == 64:
    wiki_template = "Bracket/64"
    matches_per_round = {
        1: 32,  # Round 1
        2: 16,  # Round 2
        3: 8,   # Round 3
        4: 4,   # Quarterfinals
        5: 2,   # Semifinals
        6: 1    # Grand Final
    }
    round_labels = {
        1: "Round 1",
        2: "Round 2",
        3: "Round 3",
        4: "Quarterfinals",
        5: "Semifinals",
        6: "Grand Final"
    }
elif PLAYER_COUNT == 32:
    wiki_template = "Bracket/32"
    matches_per_round = {
        1: 16,  # Round of 32
        2: 8,   # Round of 16
        3: 4,   # Quarterfinals
        4: 2,   # Semifinals
        5: 1    # Grand Final
    }
    round_labels = {
        1: "Round of 32",
        2: "Round of 16",
        3: "Quarterfinals",
        4: "Semifinals",
        5: "Grand Final"
    }
elif PLAYER_COUNT == 28:
    wiki_template = "Bracket/4L2DH8LSH8H4L2DLSL"
    matches_per_round = {
        1: 12,  # Round of 28
        2: 8,   # Round of 16
        3: 4,   # Quarterfinals
        4: 2,   # Semifinals
        5: 1    # Grand Final
    }
    round_labels = {
        1: "Round of 32",
        2: "Round of 16",
        3: "Quarterfinals",
        4: "Semifinals",
        5: "Grand Final"
    }
else:
    raise ValueError(f"Unsupported PLAYER_COUNT: {PLAYER_COUNT}. Please set to 32, 64, or 128.")

# (2) HTML Code Dump

In [10]:
html = """

"""

# (3) Execute Script

In [11]:
soup = BeautifulSoup(html, "html.parser")


# Detect round (if present)
round_name = soup.select_one(".draw-header")
round_label = round_name.text.strip() if round_name else "Round"

# Find all matches
matches = soup.select(".draw-item")

# Initialize bracket with proper format
wiki_output = [f"{{{{Bracket|{wiki_template}|id=XXXXXXXXXX"]

# Define flag replacement mappings (unchanged)
flag_replacements = {
    "cro": "hr",
    "ger": "de",
    "slo": "si",
    "uru": "uy",
    "chi": "cl", 
    "gre": "gr",
    "sui": "ch",
    "den": "dk",
    "bul": "bg",
    "por": "pt", 
    "tpe": "tw",
    "ned": "nl",
    "rsa": "za",
    "mon": "mc",
    "par": "py",
    "lat": "lv",
    "zim": "zw",
    "phi": "ph",
    "crc": "cr",
    "bar": "bb",
    # Add more as needed
    # "xxx": "yy",
}

# Convert matches into a list of processed matches (if any)
processed_matches = []

for match_index, match in enumerate(matches, start=1):
    player_links = match.select(".name a")

    bye_player = match.find("div", class_="name", string="Bye")
    if bye_player:
        continue # Skip the rest of the loop for this match
    players = []
    
    # 🔴 LOGIC BASED ON USE_FULL_NAME SETTING
    if USE_FULL_NAME:
        # Extract full name from the link's href and format it
        for link in player_links:
            href = link.get('href', '')
            full_name = format_player_name(href)
            players.append(full_name)
    else:
        # Extract abbreviated name from the link's text
        for link in player_links:
            # Original logic: 'R. Nadal <span>(1)</span>' -> 'R. Nadal'
            abbr_name = link.text.strip().split("(")[0].strip()
            players.append(abbr_name)
    
    # 🏳️ Extract flag codes (if present)
    flag_elements = match.select(".country use")
    flags = []
    for f in flag_elements:
        href = f.get("href", "")
        flag_code = ""
        if "#flag-" in href:
            flag_code = href.split("#flag-")[1].lower()
            if flag_code in flag_replacements:
                flag_code = flag_replacements[flag_code]
        flags.append(flag_code)

    # Note: ATPTour website structure often repeats score blocks for P1 and P2
    score_blocks = match.select(".scores")
    match_data = {"players": players, "flags": flags, "scores": []}

    # Assuming the first half of .scores belongs to P1 and the second half to P2
    # This requires a more robust structure check if it's an issue, but based on snippet
    # the scores are extracted correctly below from two adjacent blocks (one for P1, one for P2).
    if len(score_blocks) >= 2:
        # P1 scores are typically in the first score block, P2 in the second
        # If the input HTML structure is fixed (P1 scores, then P2 scores), this works:
        scores_p1 = [s.text.strip() for s in score_blocks[0].select(".score-item span:first-child") if s.text.strip()]
        scores_p2 = [s.text.strip() for s in score_blocks[1].select(".score-item span:first-child") if s.text.strip()]
        
        # If the structure is interlaced or different, adjust here.
        match_data["scores"] = list(zip(scores_p1, scores_p2))

    processed_matches.append(match_data)

# Generate bracket output round by round
match_counter = 0
for round_num, num_matches in matches_per_round.items():
    wiki_output.append(f"\n")
    for match_in_round in range(1, num_matches + 1):
        round_match_id = f"R{round_num}M{match_in_round}"

        if match_counter < len(processed_matches):
            m = processed_matches[match_counter]
            match_counter += 1

            p1 = m["players"][0] if len(m["players"]) > 0 else ""
            p2 = m["players"][1] if len(m["players"]) > 1 else ""

            # 🏁 Get flags (default empty if missing)
            flag1 = m["flags"][0] if len(m["flags"]) > 0 else ""
            flag2 = m["flags"][1] if len(m["flags"]) > 1 else ""

            # 🟢 Use BEST_OF_SETS setting
            match_entry = f"""|{round_match_id}={{{{Match
    |bestof={BEST_OF_SETS}
    |date=
    |opponent1={{{{SoloOpponent|{p1}|flag={flag1}}}}}
    |opponent2={{{{SoloOpponent|{p2}|flag={flag2}}}}}"""

            # Add map data - iterate up to BEST_OF_SETS
            for i in range(BEST_OF_SETS):
                if i < len(m["scores"]):
                    score1, score2 = m["scores"][i]
                    # Determine 'finished' state
                    finished = "true" if score1 and score2 else "skip"
                else:
                    score1 = score2 = ""
                    finished = "skip"
                
                match_entry += f"""
    |map{i+1}={{{{Map|map=Set {i+1}|score1={score1}|score2={score2}|finished={finished}}}}}"""

            match_entry += """
    }}"""
            wiki_output.append(match_entry)
        else:
            # Empty slot (no match data found)
            wiki_output.append(f"|{round_match_id}=")

wiki_output.append("}}") # Close the bracket

# Output result
display(HTML('''
<button onclick="navigator.clipboard.writeText(this.parentElement.querySelector('pre').innerText)" 
style="background:#4CAF50;color:white;border:none;padding:8px 12px;border-radius:6px;cursor:pointer;">
Copy code
</button>
<pre style="background:#1e1e1e;color:#dcdcdc;padding:12px;border-radius:8px;overflow-x:auto;white-space:pre-wrap;">{}</pre>
'''.format("\n".join(wiki_output))))